### Опрацювання та підготовка даних

- читаємо файли з правильним кодуванням та роздільниками
- приводимо url до нижнього регістру
- форматування акордів у вигляді "main accords: ..."
- видаляємо старі mainaccord-колонки, якщо є
- зберігаємо новий файл fra_main.csv

In [10]:
import pandas as pd
import ast
import os
from dotenv import load_dotenv
load_dotenv()
WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY")
TOKEN = os.getenv("TOKEN")

fra_perfumes = pd.read_csv("data/fra_perfumes.csv", encoding="utf-8")
fra_cleaned = pd.read_csv("data/fra_cleaned.csv", sep=";", encoding="latin1")

fra_perfumes["url"] = fra_perfumes["url"].str.lower()
fra_cleaned["url"] = fra_cleaned["url"].str.lower()

def format_accords(accord_str):
    try:
        accords = ast.literal_eval(accord_str)
        if isinstance(accords, list):
            return "main accords: " + ", ".join(accords)
    except:
        pass
    return ""

fra_perfumes["Main Accords"] = fra_perfumes["Main Accords"].apply(format_accords)
merged = fra_cleaned.merge(
    fra_perfumes[["url", "Main Accords"]],
    on="url",
    how="left"
)

columns_to_drop = ["mainaccord1", "mainaccord2", "mainaccord3", "mainaccord4", "mainaccord5"]
merged = merged.drop(columns=[c for c in columns_to_drop if c in merged.columns])


merged.to_csv("data/fra_main.csv", sep=";", index=False, encoding="utf-8")
print("збережено")

збережено


- у fra_main.csv всі колонки зʼєднати і отримуємо Description
- створюємо базове речення опису:
    1. Назва + призначення: "The {Perfume} is designed for {Gender}."
    2. Бренд, країна, рік: "Brand {Brand}, {Country}, created in {Year}."
    3. Топ ноти: "Top notes are {Top}."
    4. Середні ноти: "Middle notes are {Middle}."
    5. Базові ноти: "Base notes are {Base}."
    6. Головні акорди: "Main accords include {Main Accords}."
- зберігаємо файл як fra_working.csv

In [11]:
fra_main = pd.read_csv("data/fra_main.csv", sep=";", encoding="utf-8")

def make_description_en(row):
    parts = []
    
    if pd.notna(row["Perfume"]):
        text = f"The {row['Perfume']}"
        if pd.notna(row["Gender"]):
            text += f" is designed for {row['Gender']}"
        parts.append(text + ".")
    
    brand_country_year = []
    if pd.notna(row["Brand"]):
        brand_country_year.append(f"Brand {row['Brand']}")
    if pd.notna(row["Country"]):
        brand_country_year.append(row["Country"])
    if pd.notna(row["Year"]):
        brand_country_year.append(f"created in {int(row['Year'])}")
    if brand_country_year:
        parts.append(", ".join(brand_country_year) + ".")
    
    if pd.notna(row["Top"]):
        parts.append(f"Top notes are {row['Top']}.")
    
    if pd.notna(row["Middle"]):
        parts.append(f"Middle notes are {row['Middle']}.")
    
    if pd.notna(row["Base"]):
        parts.append(f"Base notes are {row['Base']}.")
    
    if pd.notna(row["Main Accords"]):
        accords = row["Main Accords"].replace("main accords:", "").strip()
        parts.append(f"Main accords include {accords}.")
    
    return " ".join(parts)

fra_main["Description"] = fra_main.apply(make_description_en, axis=1)
fra_main.to_csv("data/fra_working.csv", sep=";", index=False, encoding="utf-8")

print("збережено")


збережено


кінцевий файл для використання далі, залишаємо тільки url та description

In [12]:
df = pd.read_csv("data/fra_working.csv", sep=";")
df_clean = df[['url', 'Description']]
df_clean.to_csv("data/fra_final.csv", sep=";", index=False)

print("збережено")


збережено


### Перетворення у векторну датабазу
- 1 вектор на discription
- генерація векторів через ембедингову модель sentence-transformers/all-MiniLM-L6-v2 (далі буде покращено з BAAI)
- отримуємо [Вектори ароматів]
- збереження векторів у векторну базу даних weaviate

In [13]:
!pip install sentence-transformers weaviate-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [14]:
from sentence_transformers import SentenceTransformer

In [15]:
model_faster = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bf746eb2-9ce6-4ba7-b5e5-814182f9309d)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x3824c7310>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 2aaf8def-bf23-4bf3-9f28-7610dcca2dc9)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transform

In [16]:
df = pd.read_csv("data/fra_final.csv", delimiter=";")
print(df.head())

url_embeddings = model_faster.encode(df["url"].tolist(), convert_to_numpy=True)
desc_embeddings = model_faster.encode(df["Description"].tolist(), convert_to_numpy=True)

df["url_embedding"] = url_embeddings.tolist()
df["desc_embedding"] = desc_embeddings.tolist()

                                                 url  \
0  https://www.fragrantica.com/perfume/xerjoff/ac...   
1  https://www.fragrantica.com/perfume/jean-paul-...   
2  https://www.fragrantica.com/perfume/jean-paul-...   
3  https://www.fragrantica.com/perfume/bruno-bana...   
4  https://www.fragrantica.com/perfume/jean-paul-...   

                                         Description  
0  The accento-overdose-pride-edition is designed...  
1  The classique-pride-2024 is designed for women...  
2  The classique-pride-2023 is designed for unise...  
3  The pride-edition-man is designed for men. Bra...  
4  The le-male-pride-collector is designed for me...  


In [17]:
df.head()

,url,Description,url_embedding,desc_embedding
0,https://www.fragrantica.com/perfume/xerjoff/ac...,The accento-overdose-pride-edition is designed...,"[-0.04432978853583336, -0.009280821308493614, ...","[-0.03480173647403717, -0.05220318213105202, 0..."
1,https://www.fragrantica.com/perfume/jean-paul-...,The classique-pride-2024 is designed for women...,"[-0.07233963906764984, 0.03223197162151337, 0....","[-0.03264123573899269, -0.013616514392197132, ..."
2,https://www.fragrantica.com/perfume/jean-paul-...,The classique-pride-2023 is designed for unise...,"[-0.0795866847038269, 0.031219353899359703, 0....","[-0.06086832657456398, -0.002978735603392124, ..."
3,https://www.fragrantica.com/perfume/bruno-bana...,The pride-edition-man is designed for men. Bra...,"[-0.040581244975328445, 0.023872777819633484, ...","[-0.013094124384224415, 0.00038594662328250706..."
4,https://www.fragrantica.com/perfume/jean-paul-...,The le-male-pride-collector is designed for me...,"[-0.0933590978384018, 0.0480557456612587, 0.00...","[-0.05961915850639343, -0.02085299603641033, -..."


### Збереження векторів у векторну базу даних weaviate
- Підключення до Weaviate Cloud
- Створення колекції (Perfume) (+ перевірка чи існує і очищення, щоб не було збою при повторному завантаженні)
- Додавання даних у колекцію: Для кожного рядка додаємо властивості (url, description) і готовий вектор (desc_embedding)

In [18]:
pip install weaviate-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import weaviate
from weaviate.classes.init import Auth

os.environ["WEAVIATE_URL"] = WEAVIATE_URL
os.environ["WEAVIATE_API_KEY"] = WEAVIATE_API_KEY

weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    timeout_config=(10, 60),
)
print(client.is_ready())

True


In [22]:
from weaviate.classes.config import Property, DataType, Configure

if client.collections.exists("Perfume"):
    client.collections.delete("Perfume")

client.collections.create(
    name="Perfume",
    description="Perfume DB with one desc vector",
    properties=[
        Property(name="url", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
    ],
    vector_config={
        "name": "desc_embedding",
        "vectorizer": Configure.Vectorizer.none(),
        "vectorIndexConfig": Configure.VectorIndex.hnsw()._to_dict()
    }
)

print("Колекція Perfume створена!")


Колекція Perfume створена!


In [23]:
collection = client.collections.get("Perfume")

In [24]:
with collection.batch.fixed_size(100) as batch:
    for _, row in df.iterrows():
        batch.add_object(
            properties={
                "url": row["url"],
                "description": row["Description"],
            },
            vector={  
                "desc_embedding": row["desc_embedding"]
            }
        )

print("Дані додані в колекцію")

Дані додані в колекцію


In [25]:
print("Кількість об'єктів:", collection.aggregate.over_all(total_count=True).total_count)

Кількість об'єктів: 24104


### Обробка запиту користувача та формування відповіді

- Визначення запиту користувача
- Перетворення тексту на вектор (embedding з тією ж моделлю)
- Пошук найближчих топ-5 векторів у колекції до "вектора користувача": Weaviate шукає топ-5 найбільш схожих векторів у колекції за алгоритмом approximate nearest neighbor (ANN). За замовчуванням метрика схожості cosine similarity. Список вже ранжований від найбільш до менш схожого, тобто перший елемент — найбільш релевантний.
- Фільтр відповідей, якщо мінімальна схожість (certainty) менша 0.5 по cosine similarity, то цей парфум у відповідь не виводиться
- Виведення знайдених результатів, URL та Description

In [26]:
query = "vanilla floral perfume"
q_vec = model_faster.encode([query])[0].tolist()

top_n = 5
threshold = 0.5

result = collection.query.near_vector(
    near_vector=q_vec,
    target_vector="desc_embedding",
    limit=top_n,
    return_properties=["url", "description"],
    return_metadata=["distance"]
)

filtered_results = []
for obj in result.objects:
    distance = obj.metadata.distance
    if distance is not None:
        similarity = 1 - distance
        if similarity >= threshold:
            filtered_results.append((obj, similarity))

for obj, sim in filtered_results:
    print(f"URL: {obj.properties['url']}")
    print(f"Description: {obj.properties['description']}")
    print(f"Cosine similarity: {sim:.3f}")
    print("-" * 60)


URL: https://www.fragrantica.com/perfume/dame-perfumery/peach-blossom-violet-vanilla-27032.html
Description: The peach-blossom-violet-vanilla is designed for women. Brand dame-perfumery, USA, created in 2014. Top notes are peach blossom, hyacinth, red berries, bergamot. Middle notes are parma violet, jasmine, rose. Base notes are musk, tahitian vanilla, driftwood. Main accords include floral, powdery, violet, fruity, fresh spicy, fresh, musky, green, vanilla.
Cosine similarity: 0.699
------------------------------------------------------------
URL: https://www.fragrantica.com/perfume/the-body-shop/vanilla-perfume-oil-3630.html
Description: The vanilla-perfume-oil is designed for women. Brand the-body-shop, UK. Top notes are peach, lemon, apricot, plum. Middle notes are jasmine, ylang-ylang, tuberose, lily-of-the-valley, rose, orange blossom. Base notes are vanilla, amber, sandalwood, musk. Main accords include vanilla, powdery, sweet, white floral, fruity, woody, amber.
Cosine similari

Зберігання API ключа

In [27]:
os.environ["OPENROUTER_KEY"] = OPENROUTER_KEY
OPENROUTER_KEY = os.environ["OPENROUTER_KEY"]

- Готуємо запит до API (модель claude-3.5-sonnet)
- Відправляємо запит і отримуємо відповідь
- Виводимо відповідь

In [28]:
import requests

headers = {
    "Authorization": f"Bearer {OPENROUTER_KEY}",
    "Content-Type": "application/json",
}

data = {
    "model": "anthropic/claude-3.5-sonnet",
    "messages": [
        {"role": "user", "content": "Привіт, хто ти?"}
    ]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
print(response.json()["choices"][0]["message"]["content"])

Привіт! Я - Claude, AI-асистент, створений компанією Anthropic. Я намагаюсь бути корисним і давати правдиві відповіді.


### Пошуку та уточнення запиту
- Перевірка запиту на зміст
- Основна функція пошуку

- Обробляє текст користувача (query), який треба перевірити на наявність опису ароматичних нот.
- Формує запит до моделі anthropic/claude-3.5-sonnet через OpenRouter
- Відправляємо запит на сервер і чекаємо відповідь (не більше 15 секунд)
- Отримує відповідь від моделі у JSON-форматі.
- Повертає тільки текст відповіді — 'ok', якщо опис є, або уточнення українською, якщо інформації бракує. Якщо щось пішло не так (помилка мережі або API), то повертає повідомлення про помилку.

In [29]:
import requests

def check_user_query(query: str) -> str:

    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    data = {
        "model": "anthropic/claude-3.5-sonnet",
        "max_tokens": 200,
        "messages": [
            {
                "role": "system",
                "content": (
                    "Ти асистент, який перевіряє, чи містить текст опис ароматичних нот парфуму " 
                    "(наприклад: ваніль, цитрус, мускус, деревний, квітковий, фруктовий тощо). " 
                    "Якщо текст містить хоча б одну таку ноту — відповідай ТІЛЬКИ словом 'ok'. " 
                    "Якщо опис не містить ароматичних нот або інформації недостатньо — " 
                    "задай коротке уточнення українською, не більше 15 слів. " 
                    "Не пояснюй, не аналізуй, не повторюй вхідний текст."
                )
            },
            {"role": "user", "content": query},
        ],
    }

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=15
        )
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"].strip()

    except (requests.exceptions.RequestException, KeyError) as e:
        print(f"Помилка під час запиту: {e}")
        return "Не вдалося обробити запит, спробуйте ще раз."


In [30]:
test_query = "Мені хочеться щось солодке"   
result = check_user_query(test_query)
print("Claude:", result)

Claude: Які саме аромати ви маєте на увазі?


In [31]:
test_query = "Мені хочеться щось"   
result = check_user_query(test_query)
print("Claude:", result)

Claude: Опишіть які аромати вам подобаються - солодкі, свіжі, пряні?


перетворює текст користувача на вектор

In [32]:
def encode_user_query(full_query: str, model):
    vector = model.encode([full_query])[0].tolist()
    return vector

- Шукаємо у колекції Weaviate найбільш схожі вектори за полем desc_embedding до "вектору користувача"
- Обмежуємо кількість результатів топ‑k (за замовчуванням 5)
- Повертаємо список об’єктів з властивостями url і description

In [33]:
def search_in_weaviate(vector, collection, k=5):
    result = collection.query.near_vector(
        near_vector=vector,
        target_vector="desc_embedding",
        limit=k,
        return_properties=["url", "description"]
    )
    return result.objects

- Приймає знайдені результати з Weaviate і їхні описи
- Перетворює кожен опис у вектор за допомогою моделі
- Обчислює cosine similarity між вектором користувача і кожним описом
- Сортує результати за схожістю від найвищої до найнижчої.
- Повертає список об’єктів, від найрелевантнішого до найменш релевантного

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

def rerank_results(vector, results, model):
    sims = []
    for obj in results:
        desc = obj.properties["description"]
        vec = model.encode([desc])[0].reshape(1, -1)
        sim = cosine_similarity([vector], vec)[0][0]
        sims.append((sim, obj))

    sims.sort(key=lambda x: x[0], reverse=True)
    return [o for _, o in sims]


### Генерація відповіді

- Перевіряє, чи є результати для запиту
- Формує контекст із топ-K знайдених ароматів (URL + опис)
- Створює чітку RAG-підказку для моделі
- Відправляє запит до OpenRouter / Claude
- Отримує і повертає відповідь користувачу
- Обробляє помилки, якщо запит не вдався

In [35]:
def generate_response(query: str, results, model_name="anthropic/claude-3.5-sonnet", top_k=5):

    if not results:
        return "😕 Нічого не знайдено. Спробуй описати аромат інакше."

    context_docs = []
    for obj in results[:top_k]:
        url = obj.properties.get("url", "")
        desc = obj.properties.get("description", "")
        context_docs.append(f"{url}\n{desc}")

    context = "\n\n".join(context_docs)

    rag_prompt = f"""
Ти — корисний асистент, який допомагає підібрати аромат парфумів. 
Використовуй наведений контекст, щоб відповісти на запит користувача. 
Обов'язково дотримуйся правил:

1. Відповідай українською.
2. Коротко поясни, які варіанти підійшли та чому.
3. Для кожного аромату вкажи пряме посилання на аромат на сайті https://www.fragrantica.com (якщо URL є).
4. Не вигадуй аромати і не додавай неперевірену інформацію.
5. Не повторюй текст запиту і не пояснюй процес.

Контекст (URL + опис):
{context}

Запит користувача:
{query}
"""

    headers = {
        "Authorization": f"Bearer {OPENROUTER_KEY}",
        "Content-Type": "application/json",
    }

    data = {
        "model": model_name,
        "messages": [
            {"role": "system", "content": "Ти корисний ароматний асистент."},
            {"role": "user", "content": rag_prompt},
        ],
        "temperature": 0.4,
        "max_tokens": 300
    }

    try:
        r = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=data,
            timeout=15
        )
        r.raise_for_status()
        result = r.json()
        llm_answer = result["choices"][0]["message"]["content"].strip()
        return f"Відповідь:\n{llm_answer}"

    except (requests.exceptions.RequestException, KeyError) as e:
        print(f"Помилка під час запиту до LLM: {e}")
        return "Не вдалося обробити запит, спробуйте ще раз."

In [36]:
def handle_user_message(message: str, model, collection):

    clarifying = check_user_query(message)
    if clarifying.lower() != "ok":
        return clarifying

    user_vec = encode_user_query(message, model)
    raw_results = search_in_weaviate(user_vec, collection, k=5)
    ranked = rerank_results(user_vec, raw_results, model)
    answer = generate_response(message, ranked)
    return answer

print(handle_user_message("Generate something with vanilla and strawberry for me", model_faster, collection))

Відповідь:
Ось два аромати, що найкраще поєднують ваніль та полуницю:

1. Candie's Strawberry Creme
https://www.fragrantica.com/perfume/candie-s/strawberry-creme-16635.html
Солодкий десертний аромат з яскравою полуницею у верхніх нотах та теплою ваніллю і збитими вершками в базі.

2. The Body Shop Blissful Strawberry
https://www.fragrantica.com/perfume/the-body-shop/blissful-strawberry-74742.html
Більш легкий та свіжий варіант, де солодка полуниця доповнюється квітковими нотами півонії та м'яким мускусним шлейфом.


### Telegram

In [39]:
import asyncio
from aiogram import Bot, Dispatcher, Router, types
from aiogram.filters import Command
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.types import InlineKeyboardButton, InlineKeyboardMarkup

TOKEN = TOKEN

def main_menu_kb():
    return InlineKeyboardMarkup(
        inline_keyboard=[
            [
                InlineKeyboardButton(
                    text="✨ Підбір аромату за описом",
                    callback_data="search_by_description",
                )
            ],
            [
                InlineKeyboardButton(
                    text="🌷 Пошук за назвою парфуму",
                    callback_data="search_by_name",
                )
            ],
            [
                InlineKeyboardButton(
                    text="🌟 Пошук за назвою + побажання",
                    callback_data="search_by_name_with_notes",
                )
            ],
            [InlineKeyboardButton(text="Про бота", callback_data="about")],
        ]
    )

class Form(StatesGroup):
    waiting_for_description = State()
    waiting_for_name = State()
    waiting_for_name_with_notes = State()

router = Router()

@router.message(Command("start"))
async def cmd_start(message: types.Message):
    kb = InlineKeyboardMarkup(
        inline_keyboard=[
            [
                InlineKeyboardButton(
                    text="✨ Підбір аромату за описом",
                    callback_data="search_by_description",
                )
            ],
            [
                InlineKeyboardButton(
                    text="🌷 Пошук за назвою парфуму",
                    callback_data="search_by_name",
                )
            ],
            [
                InlineKeyboardButton(
                    text="🌟 Пошук за назвою + побажання",
                    callback_data="search_by_name_with_notes",
                )
            ],
            [InlineKeyboardButton(text="Про бота", callback_data="about")],
        ]
    )
    await message.answer(
        "Привіт! 👋\nЯ тестовий бот. Використай кнопки нижче або надішли /help.",
        reply_markup=kb,
    )

@router.message(Command("help"))
async def cmd_help(message: types.Message):
    await message.answer(
        "Команди:\n"
        "/start — почати\n"
        "/help — допомога\n\n"
        "Надішли будь-який текст — я відповім ехо."
    )

@router.callback_query(lambda c: c.data == "about")
async def cb_about(callback: types.CallbackQuery):
    await callback.message.answer(
        "ℹ️ Цей бот створено на aiogram 3.3.0.\nВін показує приклад кнопок і FSM."
    )
    await callback.answer()


@router.message(Form.waiting_for_name)
async def process_name(message: types.Message, state: FSMContext):
    name = message.text.strip()
    if not name:
        await message.answer("❗ Будь ласка, введи назву парфуму ще раз.")
        return
    await message.answer(f"🌷 Ось 10 парфумів, схожих на '{name}':")
    await state.clear()

@router.message(Form.waiting_for_description)
async def process_description(message: types.Message, state: FSMContext):
    description = message.text.strip()
    data = await state.get_data()
    attempts = data.get("attempts", 0)
    base_prompt = data.get("base_prompt", "")
    print(base_prompt)
    if attempts == 0 and not base_prompt:
        base_prompt = description
        print(base_prompt)
    clarifying = check_user_query(description)
    if clarifying.lower() != "ok":
        attempts += 1
        await state.update_data(attempts=attempts, base_prompt=base_prompt)

        if attempts >= 3:
            await message.answer("⚠️ Можливо, спробуй переформулювати запит пізніше.")
            await message.answer("Оберіть наступну дію:", reply_markup=main_menu_kb())
            await state.clear()
            return

        await message.answer(
            "🧐 Опиши аромат детальніше — додай нотки (ваніль, мускус, жасмин тощо)."
        )
        return

    full_prompt = f"{base_prompt}, {description}" if base_prompt else description
    print(full_prompt)
    recs = handle_user_message(full_prompt, model_faster, collection)
    await message.answer(recs)
    await message.answer("Оберіть наступну дію:", reply_markup=main_menu_kb())
    await state.clear()

@router.message()
async def echo(message: types.Message):
    if message.text and not message.text.startswith("/"):
        await message.answer(f"Ехо: {message.text}")
    else:
        await message.answer("Невідома команда. Спробуй /help")

@router.callback_query(lambda c: c.data == "search_by_description")
async def cb_search_description(callback: types.CallbackQuery, state: FSMContext):
    await callback.message.answer("Введи опис бажаного аромату:")
    await state.set_state(Form.waiting_for_description)  # бот чекає опис
    await callback.answer()

@router.callback_query(lambda c: c.data == "search_by_name")
async def cb_search_name(callback: types.CallbackQuery):
    await callback.message.answer("Введи назву парфуму:")
    await callback.answer()

@router.callback_query(lambda c: c.data == "search_by_name_with_notes")
async def cb_search_name_with_notes(callback: types.CallbackQuery):
    await callback.message.answer(
        "Введи назву парфуму, а потім додаткові нотки, які хочеш додати чи виключити:"
    )
    await callback.answer()

async def main():
    bot = Bot(token=TOKEN)
    dp = Dispatcher(storage=MemoryStorage())
    dp.include_router(router)
    print("✅ Бот запущений і працює!")
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()

✅ Бот запущений і працює!

Хочу щось свіже
Хочу щось свіже
Хочу щось свіже, Хочу аромат з лимоном, жасмином та зеленим чаєм

Хочу відчувати ноти шкіри та дерева
Хочу відчувати ноти шкіри та дерева, Хочу відчувати ноти шкіри та дерева

I would like something with top vanilla notes and milk chocolate
I would like something with top vanilla notes and milk chocolate, I would like something with top vanilla notes and milk chocolate

I would like something with pepper and citrus
I would like something with pepper and citrus, I would like something with pepper and citrus

Хочу підібрати подрузі аромат із запахом молочної каші
Хочу підібрати подрузі аромат із запахом молочної каші


Received SIGINT signal
